In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
!pip install webdriver-manager


In [9]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

from time import sleep

from dotenv import load_dotenv
load_dotenv()

False

In [10]:
# if you don't have a .env file, you can create one and add the following lines:
os.environ['EMAIL'] = 'daziaconsulting@gmail.com'
os.environ['PASSWORD'] = 'Dazia@2024'   

In [11]:
import requests

# API base URL
url = "https://api.reversecontact.com/enrichment"

# Get user input for email
# email = input("Enter the email address: ")
email = "meetpatel8122001@gmail.com"

# Query parameters including the API key
querystring = {
    "apikey": "sk_b15fdcd5bf159cd1aa6ca2673a28d61de5454ddf",
    "email": email
}

# Make the GET request
response = requests.get(url, params=querystring)

# Check the response
if response.status_code == 200:
    data = response.json()
    if data.get("success"):
        linkedin_url = data["person"].get("linkedInUrl")
        if linkedin_url:
            print(f"LinkedIn URL: {linkedin_url}")
        else:
            print("LinkedIn URL not found.")
    else:
        print("API call unsuccessful. Check the provided email.")
else:
    print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")
    print(f"Response: {response.text}")


Failed to fetch data. HTTP Status Code: 404
Response: {"success":false,"title":"Not Found","msg":"No data found for this person."}


In [12]:
linkedin_url

NameError: name 'linkedin_url' is not defined

In [ ]:
driver = webdriver.Chrome()

In [ ]:
driver.get('https://www.linkedin.com/login')

In [ ]:
driver.title

'LinkedIn Login, Sign in | LinkedIn'

In [ ]:
email = driver.find_element(By.ID, 'username')
email.send_keys(os.environ['EMAIL'])

password = driver.find_element(By.ID, 'password')
password.send_keys(os.environ['PASSWORD'])

password.submit()

In [ ]:
## MAKE SURE TO USE ONLY THIS URL TO AVOID BEING STUCK IN ERRORS

# url = "https://www.linkedin.com/in/hannah-nardone-126655282/"
# url = "https://www.linkedin.com/in/badshah/"
url = linkedin_url
driver.get(url)

In [ ]:
profile_data = {}

In [ ]:
driver.title

'(27) Rebecca Chen | LinkedIn'

In [ ]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

name = soup.find('h1')

name = name.get_text().strip()

profile_data['name'] = name
profile_data['url'] = url

In [ ]:
profile_data

{'name': 'Rebecca Chen',
 'url': 'https://www.linkedin.com/in/rebecca-chen-0120251b0'}

In [ ]:
headline = soup.find('div', {'class': 'text-body-medium break-words'})
headline = headline.get_text().strip()

profile_data['headline'] = headline

In [ ]:
profile_data

{'name': 'Rebecca Chen',
 'url': 'https://www.linkedin.com/in/rebecca-chen-0120251b0',
 'headline': 'Strategic Talent Acquisition Consultant'}

In [ ]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

about = soup.find('div', {'class': 'display-flex ph5 pv3'})

if about:
    about = about.get_text().strip()
else:
    about = None

profile_data['about'] = about

In [ ]:
profile_data

{'name': 'Rebecca Chen',
 'url': 'https://www.linkedin.com/in/rebecca-chen-0120251b0',
 'headline': 'Strategic Talent Acquisition Consultant',
 'about': None}

In [ ]:
url2 = f"{url}/overlay/contact-info/"
driver.get(url2)

In [ ]:
# page_source = driver.page_source
# soup = BeautifulSoup(page_source, 'lxml')


# email = soup.find('div', {'class': 'sTcdifezBUxpROXbnKHLDEcFBCqbbiGT link-without-visited-state t-14'})

# if email:
#     email = email.get_text().strip()
# else:
#     print("blank")

# profile_data['email'] = email

In [ ]:
profile_data['email'] = email

In [ ]:
profile_data

{'name': 'Rebecca Chen',
 'url': 'https://www.linkedin.com/in/rebecca-chen-0120251b0',
 'headline': 'Strategic Talent Acquisition Consultant',
 'about': None,
 'email': <selenium.webdriver.remote.webelement.WebElement (session="e4baac9ea7856d97ddb107528b10933b", element="f.C3F14D6989D1BCABEE054C082D9E0A7E.d.DC835715531DB06F6406AEB3AD7FA5DB.e.1")>}

In [ ]:
import os
from groq import Groq

# Initialize Groq API
os.environ["GROQ_API_KEY"] = "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"
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

def extract_dynamic_tags_and_summarize(input_data):
    """
    Extract dynamic tags based on the understanding of the AI model
    and summarize the 'about' section in 50 words.
    """
    try:
        # Extract headline and about section
        headline = input_data.get("headline", "")
        about = input_data.get("about", "")

        # AI prompt to extract dynamic tags
        tag_prompt = f"""
        Analyze the following headline and about section to extract key tags that best describe the person's background, roles, interests, and areas of expertise. Only in Tags format like McMaster University, Mathematics, Student etc.
        
        Headline: {headline}
        About: {about}
        
        Extract the tags dynamically and provide them as a python list of strings.
        """
        tag_response = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[
                {"role": "system", "content": "You are an expert AI for analyzing text and extracting meaningful tags."},
                {"role": "user", "content": tag_prompt}
            ]
        )
        tags = tag_response.choices[0].message.content.strip()
# tag_response.choices[0].message.content.strip()
        # AI prompt to summarize the 'about' section
        summary_prompt = f"""
        Summarize the following text into 50 words:
        "{about}"
        """
        summary_response = client.chat.completions.create(
            model="llama3-8b-8192",

            messages=[
                {"role": "system", "content": "You are a text summarization assistant."},
                {"role": "user", "content": summary_prompt}
            ]
        )
        summary = summary_response.choices[0].message.content.strip()
# summary_response.choices[0].message.content.strip()
        return {
            "tags": tags,
            "summary": summary
        }

    except Exception as e:
        return {"error": str(e)}

# Input data
input_data = {'name': 'Shabs FindGrant',
 'url': 'https://www.linkedin.com/in/badshah/',
 'headline': 'FindGrant.ca | AI for finding grants',
 'about': "Use #AI to help you find over $37 billion in #grants from the federal, provincial and city governments.. Check out https://FindGrant.ca! I have 15+ years experience in IT and Finance Specialities:  Prototypes, Blockchain, Full stack development, Responsive web frameworks, People Management, Information Technology, CommunityWinner at Blockhack 2023 (Grand Prize), Elevate Toronto 2018 (First place), HackJustice 2017 (Second place), Climathon Toronto 2016 (People's Choice), HorseRacing 3.0 2016 (2nd Grand Prize)Use #AI to help you find over $37 billion in #grants from the federal, provincial and city governments.. Check out https://FindGrant.ca! \n\nI have 15+ years experience in IT and Finance \nSpecialities:  Prototypes, Blockchain, Full stack development, Responsive web frameworks, People Management, Information Technology, Community\n\nWinner at Blockhack 2023 (Grand Prize), Elevate Toronto 2018 (First place), \nHackJustice 2017 (Second place), Climathon Toronto 2016 (People's Choice), HorseRacing 3.0 2016 (2nd Grand Prize)",
 'email': None}

# Extract dynamic tags and summarize
result = extract_dynamic_tags_and_summarize(input_data)
print(result)

{'tags': "After analyzing the provided headline and about section, I extracted the following tags that best describe the person's background, roles, interests, and areas of expertise:\n\n`['FindGrant.ca', 'AI', 'Grant', 'IT', 'Finance', 'Prototypes', 'Blockchain', 'Full_stack_development', 'Responsive_web_frameworks', 'People_Management', 'Information_Technology', 'Community', 'Blockhack', 'Elevate_Toronto', 'HackJustice', 'Climathon_Toronto', 'Horse_Racing_3.0', '15+ years_experience']`\n\nThese tags capture the person's background in IT and Finance, their specializations in Prototypes, Blockchain, Full-stack development, and Responsive web frameworks, as well as their experience in People Management and Information Technology. The tags also highlight their achievements in various hackathons and competitions, demonstrating their expertise in these areas. Finally, the presence of the specific grant platform FindGrant.ca and the emphasis on AI suggest that the person's primary area of e

## New code

In [ ]:
import os
from groq import Groq

# Initialize Groq API
os.environ["GROQ_API_KEY"] = "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"
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

def extract_tags_summarize_and_create_note(input_data):
    """
    Extract dynamic tags, summarize the 'about' section in 50 words as "I",
    and create a 100-word LinkedIn connection request note.
    """
    try:
        # Extract relevant fields
        name = input_data.get("name", "")
        headline = input_data.get("headline", "")
        # about = input_data.get("about", "")

        # AI prompt to extract dynamic tags
        tag_prompt = f"""
        Analyze the following information and extract key tags that describe the person's roles, interests, and expertise.
        Provide ONLY a comma-separated list of tags. Do NOT include any additional text or explanations.

        Name: {name}
        Headline: {headline}
        About: {about}
        """
        tag_response = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[
                {"role": "system", "content": "You are an expert AI for analyzing text and extracting meaningful tags."},
                {"role": "user", "content": tag_prompt}
            ]
        )
        tags = tag_response.choices[0].message.content.strip()

        # AI prompt to summarize the 'about' section in 50 words as "I"
        summary_prompt = f"""
        Summarize the following text into 50 words in first person ("I"). 
        Provide ONLY the summary.
        Strongly Do NOT include any additional text or explanations.

        About: "{about}"
        """
        summary_response = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[
                {"role": "system", "content": "You are a text summarization assistant."},
                {"role": "user", "content": summary_prompt}
            ]
        )
        summary = summary_response.choices[0].message.content.strip()

        # AI prompt to create a 100-word LinkedIn connection request note
        note_prompt = f"""
        Create a 100-word LinkedIn connection request note based on the following information.
        Provide ONLY the note.
        Strongly Do NOT include any additional text or explanations.

        Name: {name}
        Headline: {headline}
        About: {about}
        """
        note_response = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[
                {"role": "system", "content": "You are a professional networking assistant."},
                {"role": "user", "content": note_prompt}
            ]
        )
        note = note_response.choices[0].message.content.strip()

        return {
            "tags": tags,
            "summary": summary,
            "linkedin_note": note
        }

    except Exception as e:
        return {"error": str(e)}

# Input data
input_data = profile_data
# input_data = {
#     'name': 'Shabs FindGrant',
#     'url': 'https://www.linkedin.com/in/badshah/',
#     'headline': 'FindGrant.ca | AI for finding grants',
#     'about': "Use #AI to help you find over $37 billion in #grants from the federal, provincial and city governments.. Check out https://FindGrant.ca! I have 15+ years experience in IT and Finance Specialities:  Prototypes, Blockchain, Full stack development, Responsive web frameworks, People Management, Information Technology, CommunityWinner at Blockhack 2023 (Grand Prize), Elevate Toronto 2018 (First place), HackJustice 2017 (Second place), Climathon Toronto 2016 (People's Choice), HorseRacing 3.0 2016 (2nd Grand Prize)Use #AI to help you find over $37 billion in #grants from the federal, provincial and city governments.. Check out https://FindGrant.ca! \n\nI have 15+ years experience in IT and Finance \nSpecialities:  Prototypes, Blockchain, Full stack development, Responsive web frameworks, People Management, Information Technology, Community\n\nWinner at Blockhack 2023 (Grand Prize), Elevate Toronto 2018 (First place), \nHackJustice 2017 (Second place), Climathon Toronto 2016 (People's Choice), HorseRacing 3.0 2016 (2nd Grand Prize)",
#     'email': None
# }

# Extract tags, summarize, and create LinkedIn note
result = extract_tags_summarize_and_create_note(input_data)

In [ ]:
result

{'tags': 'Strategic Talent Acquisition Consultant, Recruitment Specialist, Human Resources, Talent Management',
 'summary': "I'm exploring a new area of natural language processing, focusing on text summarization. I analyze large amounts of text data, identifying key points and condensing them into a concise summary. My goal is to provide accurate and understandable summaries, helping users quickly grasp complex information.",
 'linkedin_note': '"Hi Rebecca, I came across your profile and was impressed with your experience as a Strategic Talent Acquisition Consultant. I\'d love to connect with you on LinkedIn as I\'m always looking to expand my network in the talent acquisition space. I\'m a professional networking assistant, dedicated to helping professionals like yourself build meaningful connections and opportunities. I\'d be happy to learn more about your work and share my own expertise in return. Looking forward to hearing from you!"'}